<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f96dea1882f9bcae1a5a12ec5d56fa3ab76b1112fd7a545e653320ddbc225e6a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-01 11:17:31
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: 7.79 K (0.05%)
Current PnL: -22.59 L (-14.81%)
CY Booked + Current PnL: -8.70 L (-5.7%)
-------------------
Total profit:  1.63 L
Total loss:  -24.22 L
-------------------
Total Booked + Current PnL: 18.28 L (14.55%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.7%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.00 L (61.46%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.39%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.16,-8.67,9.51,0.02,15162.0,-15132.0,159427.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.11,38.67,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.60,-16.83,20.34,0.09,17049.0,-16957.0,83820.0,94.45,36.0,M-SC,3.20,253.0,-0.99,0.59,10.12,OX40N,NTT,DURABLES
43,ITC,452.00,-0.27,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,44.0,X-LC,1.33,5.0,-0.13,1.38,3.98,X40,NTT,FMCG
83,VOLTAS,1530.00,-0.68,6.94,11.92,19.69,24442.0,13308.0,205050.0,-3.43,48.0,X-MC,3.23,78.0,0.54,1.43,14.21,XY25,NTT,AC
51,MASFIN,398.61,1.75,-3.80,26.87,22.05,25328.0,-3720.0,94260.0,-16.89,55.0,H-SC,6.89,164.0,-0.15,0.66,37.01,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,5.00,-44.97,216.47,74.15,194892.0,-73573.0,90032.0,-262.20,38.0,H-SC,18.15,149.0,-0.38,0.63,15.41,XR,NTT,CHEMICALS
37,IEX,219.00,4.82,-0.92,50.00,48.63,99061.0,-1832.0,198122.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.38,10.53,XR,NTT,MISC
15,CAMS,4762.00,1.90,3.35,20.74,24.79,53987.0,8448.0,260304.0,-1.47,54.0,X-SC,1.89,86.0,0.16,1.82,27.27,X40N,NTT,MISC
70,SURYODAY,216.00,1.80,-18.06,50.16,23.03,73598.0,-32344.0,146727.0,60.09,52.0,H-SC,9.73,167.0,-0.44,1.02,45.58,XR,NTT,BANKS
51,MASFIN,398.61,1.75,-3.80,26.87,22.05,25328.0,-3720.0,94260.0,-16.89,55.0,H-SC,6.89,164.0,-0.15,0.66,37.01,XR,ATH,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-4.47,-17.92,121.64,81.92,114617.0,-20572.0,94226.0,-51.94,21.0,M-SC,2.89,236.0,-0.18,0.66,11.55,XR,NTT,DURABLES
54,PAGEIND,51605.08,-1.77,-8.05,37.10,26.07,55858.0,-13180.0,150560.0,-32.28,29.0,X-MC,9.91,55.0,-0.24,1.05,0.00,X40,ATH,APPARELS
27,GLAXO,3466.20,-1.73,1.36,37.28,39.15,75306.0,2716.0,202000.0,-18.56,44.0,X-MC,10.51,60.0,0.04,1.41,30.76,X40,ATH,PHARMA
2,ABBOTINDIA,35195.00,-1.41,-1.07,18.70,17.44,27723.0,-1597.0,148250.0,-14.73,49.0,X-MC,4.02,63.0,-0.06,1.04,17.89,X40,ATH,PHARMA
28,GREENPANEL,537.00,-1.38,-34.49,118.51,43.14,142124.0,-63152.0,119926.0,168.73,28.0,M-SC,6.32,240.0,-0.44,0.84,12.81,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.48,-0.26,107.26,106.72,131944.0,-323.0,123013.0,-50.63,38.0,H-SC,2.38,139.0,-0.00,0.86,23.05,AR,ATH,MISC
37,IEX,219.00,4.82,-0.92,50.00,48.63,99061.0,-1832.0,198122.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.38,10.53,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.71,-4.44,117.57,107.91,165882.0,-6552.0,141092.0,-22.17,39.0,M-SC,10.82,216.0,-0.04,0.99,0.89,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.16,-8.67,9.51,0.02,15162.0,-15132.0,159427.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.11,38.67,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.60,-16.83,20.34,0.09,17049.0,-16957.0,83820.0,94.45,36.0,M-SC,3.20,253.0,-0.99,0.59,10.12,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.27,-21.29,44.17,13.47,93753.0,-57411.0,212256.0,-67.73,38.0,H-SC,1.73,158.0,-0.61,1.48,7.93,XY24,NTT,PAINTS
67,SIS,528.00,-0.80,-24.99,61.67,21.26,51361.0,-27749.0,83283.0,1963.73,44.0,H-SC,3.71,166.0,-0.54,0.58,12.52,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,0.08,2.42,68.19,72.27,118347.0,4107.0,173555.0,-11.98,59.0,M-LC,3.51,99.0,0.03,1.21,8.75,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.08,2.42,68.19,72.27,118347.0,4107.0,173555.0,-11.98,59.0,M-LC,3.51,99.0,0.03,1.21,8.75,XR,NTT,IT
38,INDIAMART,4810.62,0.48,-0.26,107.26,106.72,131944.0,-323.0,123013.0,-50.63,38.0,H-SC,2.38,139.0,-0.00,0.86,23.05,AR,ATH,MISC
37,IEX,219.00,4.82,-0.92,50.00,48.63,99061.0,-1832.0,198122.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.38,10.53,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,-0.67,-1.28,37.38,35.63,77339.0,-2674.0,206900.0,-15.41,41.0,H-MC,3.70,119.0,-0.03,1.44,15.18,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.71,-4.44,117.57,107.91,165882.0,-6552.0,141092.0,-22.17,39.0,M-SC,10.82,216.0,-0.04,0.99,0.89,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.22,-36.33,112.36,35.20,92046.0,-46749.0,81921.0,7.44,21.0,X-SC,13.63,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.49,-14.49,73.60,48.45,126196.0,-29044.0,171462.0,-20.73,28.0,X-MC,7.77,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-1.77,-8.05,37.10,26.07,55858.0,-13180.0,150560.0,-32.28,29.0,X-MC,9.91,55.0,-0.24,1.05,0.00,X40,ATH,APPARELS
76,TMPV,600.00,1.22,-14.42,66.14,42.18,156456.0,-39857.0,236553.0,-23.08,33.0,X-LC,2.32,3.0,-0.25,1.65,2.47,XY24,NTT,AUTO
22,DIXON,18931.72,-0.21,-6.43,29.94,21.58,39260.0,-9009.0,131130.0,-52.80,35.0,X-MC,6.23,56.0,-0.23,0.92,16.09,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.27,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,44.0,X-LC,1.33,5.0,-0.13,1.38,3.98,X40,NTT,FMCG
3,ACC,3906.0,0.16,-22.11,110.74,64.15,205257.0,-52601.0,185350.0,-54.57,49.0,X-SC,1.56,82.0,-0.26,1.29,3.97,XY24,BTT,CEMENT
15,CAMS,4762.0,1.90,3.35,20.74,24.79,53987.0,8448.0,260304.0,-1.47,54.0,X-SC,1.89,86.0,0.16,1.82,27.27,X40N,NTT,MISC
66,SIEMENS,4671.5,-0.07,-11.48,41.78,25.51,68828.0,-21355.0,164740.0,1.14,63.0,H-LC,2.02,49.0,-0.31,1.15,21.40,AR,ATH,ELECTRICAL
36,ICICIPRULI,790.0,0.20,2.95,27.21,30.97,49509.0,5215.0,181953.0,-21.26,55.0,X-MC,2.12,75.0,0.11,1.27,15.88,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.22,-36.33,112.36,35.20,92046.0,-46749.0,81921.0,7.44,21.0,X-SC,13.63,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.49,-14.49,73.60,48.45,126196.0,-29044.0,171462.0,-20.73,28.0,X-MC,7.77,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-1.77,-8.05,37.10,26.07,55858.0,-13180.0,150560.0,-32.28,29.0,X-MC,9.91,55.0,-0.24,1.05,0.00,X40,ATH,APPARELS
55,PGHH,17907.65,0.49,-4.25,39.70,33.76,76337.0,-8535.0,192285.0,-32.51,36.0,X-MC,4.05,57.0,-0.11,1.34,0.73,X40,ATH,FMCG
19,COLPAL,3726.84,-0.61,-18.16,72.91,41.51,157150.0,-47825.0,215540.0,-2.56,40.0,X-MC,7.22,61.0,-0.30,1.51,1.15,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,0.48,-27.15,96.08,42.84,45501.0,-17649.0,47357.0,-52.07,43.0,X-SC,37.48,83.0,-0.39,0.33,3.05,XY24,NTT,MISC
60,RELAXO,1176.00,0.07,-46.96,187.14,52.31,144088.0,-68165.0,76995.0,-43.57,38.0,X-SC,4.80,91.0,-0.47,0.54,3.06,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.22,-36.33,112.36,35.20,92046.0,-46749.0,81921.0,7.44,21.0,X-SC,13.63,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.21,-6.43,29.94,21.58,39260.0,-9009.0,131130.0,-52.80,35.0,X-MC,6.23,56.0,-0.23,0.92,16.09,X40N,ATH,IT
52,MEDANTA,1486.00,-0.64,3.37,20.03,24.08,26285.0,4278.0,131228.0,-5.01,46.0,X-SC,5.95,89.0,0.16,0.92,23.65,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.13,-14.86,40.10,19.28,118114.0,-51407.0,294549.0,-25.89,57.0,X-LC,7.02,1.0,-0.44,2.06,8.89,X40,ATH,IT
41,INFY,2275.00,0.05,6.26,45.75,54.87,152106.0,19584.0,332472.0,-17.09,61.0,X-LC,2.29,2.0,0.13,2.32,15.06,X40,BTT,IT
76,TMPV,600.00,1.22,-14.42,66.14,42.18,156456.0,-39857.0,236553.0,-23.08,33.0,X-LC,2.32,3.0,-0.25,1.65,2.47,XY24,NTT,AUTO
82,VBL,671.64,0.63,-2.11,38.60,35.67,119342.0,-6667.0,309175.0,-13.74,64.0,X-LC,5.48,4.0,-0.06,2.16,11.45,X40N,ATH,FMCG
43,ITC,452.00,-0.27,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,44.0,X-LC,1.33,5.0,-0.13,1.38,3.98,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.60,-36.99,110.92,32.89,53956.0,-28562.0,48644.0,-700.55,63.0,L-MC,5.82,259.0,-0.53,0.34,33.96,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.25,-13.98,111.19,81.67,89728.0,-13112.0,80698.0,7255.56,47.0,L-SC,19.39,271.0,-0.15,0.56,56.28,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.54,-11.95,58.47,39.54,52082.0,-12084.0,89075.0,-31.16,65.0,M-SC,6.69,220.0,-0.23,0.62,25.60,AR,ATH,AUTO
51,MASFIN,398.61,1.75,-3.80,26.87,22.05,25328.0,-3720.0,94260.0,-16.89,55.0,H-SC,6.89,164.0,-0.15,0.66,37.01,XR,ATH,FINANCE
70,SURYODAY,216.00,1.80,-18.06,50.16,23.03,73598.0,-32344.0,146727.0,60.09,52.0,H-SC,9.73,167.0,-0.44,1.02,45.58,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.16,-8.67,9.51,0.02,15162.0,-15132.0,159427.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.11,38.67,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.52,-0.06,21.01,20.95,52867.0,-141.0,251627.0,-1.05,71.0,X-LC,15.75,31.0,-0.00,1.76,35.56,X40,ATH,PAINTS
50,LTIM,7201.88,0.52,12.04,17.51,31.66,48289.0,29631.0,275782.0,7.74,68.0,H-LC,13.96,43.0,0.61,1.93,53.02,X200,ATH,IT
40,INDUSINDBK,1800.00,-0.60,-36.99,110.92,32.89,53956.0,-28562.0,48644.0,-700.55,63.0,L-MC,5.82,259.0,-0.53,0.34,33.96,XR,NTT,BANKS
0,5PAISA,593.00,-0.31,-36.43,77.60,12.90,107788.0,-79594.0,138902.0,104.33,54.0,H-SC,12.81,161.0,-0.74,0.97,15.50,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.96
1,25,44.86
2,50,75.72


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.50
MC    29.11
LC    25.41
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.78
X40      21.21
X40N     12.05
XR        9.62
XY25      9.09
AR        9.03
OX40N     7.59
X200      1.93
SR        0.97
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.93
X-MC    22.64
X-LC    20.15
M-SC    11.89
X-SC     8.29
H-MC     4.75
H-LC     3.08
L-SC     1.39
M-MC     1.38
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.86,-4.31,38.04
IT,13.81,-14.38,73.43
FINANCE,9.49,-14.07,62.26
MISC,7.41,-22.43,72.59
ELECTRICAL,6.00,-9.31,49.35
PAINTS,5.94,-8.57,24.70
INSURANCE,4.40,-0.34,35.70
PHARMA,3.89,-0.28,32.37
AUTO,3.39,-18.08,68.72


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3058351.0,21
XR,1284729.0,13
AR,1278892.0,10
X40,1003128.0,14
X40N,725370.0,9
OX40N,704783.0,10
XY25,339517.0,6
SR,280960.0,2
MH,76404.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3549239.0,25
M-SC,1357800.0,15
X-MC,1243789.0,16
X-LC,878548.0,11
X-SC,733818.0,8
H-MC,398729.0,3
L-SC,261741.0,3
M-LC,118347.0,1
H-LC,117117.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1211540.0      6
           AR         880126.0      5
M-SC       XY24       782058.0      6
H-SC       XR         774292.0      7
X-MC       X40        442518.0      7
X-LC       X40        392359.0      5
X-MC       XY24       372404.0      3
H-SC       OX40N      325917.0      4
M-SC       OX40N      291714.0      5
X-SC       X40N       288524.0      3
H-SC       SR         280960.0      2
X-SC       XY24       277043.0      3
X-MC       X40N       247275.0      4
X-LC       XY24       226032.0      2
H-MC       AR         209455.0      2
X-LC       X40N       189571.0      2
H-MC       XY24       189274.0      1
X-MC       XY25       181592.0      2
L-SC       XR         174589.0      2
X-SC       X40        168251.0      2
M-SC       XR         163545.0      2
           AR         120483.0      2
M-LC       XR         118347.0      1
L-SC       OX40N       87152.0      1
H-SC       MH          76404.0      1
X-LC       XY25        70586.0      2
H-LC       AR          68828.0      1
L-MC       XR          53956.0      1
H-LC       X200        48289.0      1
M-MC       XY25        47269.0      1
L-LC       XY25        40070.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
